In [4]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
%matplotlib inline

In [5]:
p_path="/home/takeyama/pywork/ipython/2016-05-16/"

In [6]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


In [9]:
class RecordData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._AccX=np.array([])
        self._AccY=np.array([])
        self._AccZ=np.array([])
        self._GyrX=np.array([])
        self._GyrY=np.array([])
        self._GyrZ=np.array([])
        self._Columns=np.array([])
        
        # fft data
        self._fft_AccX=np.array([])
        self._fft_AccY=np.array([])
        self._fft_AccZ=np.array([])
        self._fft_GyrX=np.array([])
        self._fft_GyrY=np.array([])
        self._fft_GyrZ=np.array([])

        # power spectol data
        self._power_AccX=np.array([])
        self._power_AccY=np.array([])
        self._power_AccZ=np.array([])
        self._power_GyrX=np.array([])
        self._power_GyrY=np.array([])
        self._power_GyrZ=np.array([])
        
    def ImportCSV(self,csv_file):
        # design dataframe
        self._data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        self._data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        self._data.Time=pd.to_datetime(self._data.Time)
        self._data = pd.pivot_table(self._data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        self._AccX=self._data.AccX.values
        self._AccY=self._data.AccY.values
        self._AccZ=self._data.AccZ.values
        self._GyrX=self._data.GyrX.values
        self._GyrY=self._data.GyrY.values
        self._GyrZ=self._data.GyrZ.values
        self._Time=self._data.index.to_pydatetime().astype('datetime64[ns]')
        self._Columns=['Time','AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
    
    def GetColumns(self):
        return self._Columns

    def ShowAllDf(self):
        print self._AccX;print self._AccY;print self._AccZ;print self._GyrX;print self._GyrY;print self._GyrZ;print self._Time
        
    def _String2Var(self,Sname):
        if(Sname=='AccX'):return self._AccX
        elif(Sname=='AccY'):return self._AccY
        elif(Sname=='AccZ'):return self._AccZ
        elif(Sname=='GyrX'):return self._GyrX
        elif(Sname=='GyrY'):return self._GyrY
        elif(Sname=='GyrZ'):return self._GyrZ
        elif(Sname=='Time'):return self._Time
        else:print 'Wrong Axis Name'
    
    def _Time2Num(self,time):
        index=np.where(test._Time==np.datetime64(time) )
        return index[0][0]    
    
    def _Query(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        return data[rng[0]:rng[1]]
            
    def GetLength(self,Sname):
        data = _String2Var(Sname)
        print Sname+' length ='+str( len(data) )
            
    def ShowQuery(self,Sname,rng=[]):
        data = self._String2Var(Sname)
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._Time[0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._Query(Sname,[start,goal])
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._Query(Sname,[start,goal])
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break   
    def _set_fft(self,Sname,array):
        #print array
        if(Sname=='fft_AccX'):   self._fft_AccX=np.append(self._fft_AccX,np.array([array]) )
        elif(Sname=='fft_AccY'): self._fft_AccY=np.append(self._fft_AccY,np.array([array]) )
        elif(Sname=='fft_AccZ'): self._fft_AccZ=np.append(self._fft_AccZ,np.array([array]) )
        elif(Sname=='fft_GyrX'): self._fft_GyrX=np.append(self._fft_GyrX,np.array([array]) )
        elif(Sname=='fft_GyrY'): self._fft_GyrY=np.append(self._fft_GyrY,np.array([array]) )
        elif(Sname=='fft_GyrZ'): self._fft_GyrZ=np.append(self._fft_GyrZ,np.array([array]) )
        else:print 'Wrong Axis Name'
            
    def _GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def _GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        for n,f in zip( self._Columns[1:],self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            count=0
            while True:
                try:
                    #print 'count ='+str(count)
                    d=sw.next()
                    # fftの直流成分を除くsample/2のみ
                    self._set_fft(f,fftpack.fft(d)[1:(samp/2)+1])     
                    count = count+1
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    break   
                    
        self._fft_AccX=self._fft_AccX.reshape(len(self._fft_AccX)/(samp/2),(samp/2) )
        self._fft_AccY=self._fft_AccY.reshape(len(self._fft_AccY)/(samp/2),(samp/2) )
        self._fft_AccZ=self._fft_AccZ.reshape(len(self._fft_AccZ)/(samp/2),(samp/2) )
        self._fft_GyrX=self._fft_GyrX.reshape(len(self._fft_GyrX)/(samp/2),(samp/2) )
        self._fft_GyrY=self._fft_GyrY.reshape(len(self._fft_GyrY)/(samp/2),(samp/2) )
        self._fft_GyrZ=self._fft_GyrZ.reshape(len(self._fft_GyrZ)/(samp/2),(samp/2) )
        
        np.savez(p_path+'fft/'+'fft_AccX_'+str(samp),self._fft_AccX)
        np.savez(p_path+'fft/'+'fft_AccY_'+str(samp),self._fft_AccY)
        np.savez(p_path+'fft/'+'fft_AccZ_'+str(samp),self._fft_AccZ)
        np.savez(p_path+'fft/'+'fft_GyrX_'+str(samp),self._fft_GyrX)
        np.savez(p_path+'fft/'+'fft_GyrY_'+str(samp),self._fft_GyrY)
        np.savez(p_path+'fft/'+'fft_GyrZ_'+str(samp),self._fft_GyrZ)

        
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        local_fft_AccX=np.load(p_path+'fft/'+'fft_AccX_'+str(samp)+'.npz')['arr_0']
        local_fft_AccY=np.load(p_path+'fft/'+'fft_AccY_'+str(samp)+'.npz')['arr_0']
        local_fft_AccZ=np.load(p_path+'fft/'+'fft_AccZ_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrX=np.load(p_path+'fft/'+'fft_GyrX_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrY=np.load(p_path+'fft/'+'fft_GyrY_'+str(samp)+'.npz')['arr_0']
        local_fft_GyrZ=np.load(p_path+'fft/'+'fft_GyrZ_'+str(samp)+'.npz')['arr_0']
           
        self._power_AccX = self._power(local_fft_AccX)
        self._power_AccY = self._power(local_fft_AccY)
        self._power_AccZ = self._power(local_fft_AccZ)
        self._power_GyrX = self._power(local_fft_GyrX)
        self._power_GyrY = self._power(local_fft_GyrY)
        self._power_GyrZ = self._power(local_fft_GyrZ)

        np.savez(p_path+'power/'+'power_AccX_'+str(samp),self._power_AccX)
        np.savez(p_path+'power/'+'power_AccY_'+str(samp),self._power_AccY)
        np.savez(p_path+'power/'+'power_AccZ_'+str(samp),self._power_AccZ)
        np.savez(p_path+'power/'+'power_GyrX_'+str(samp),self._power_GyrX)
        np.savez(p_path+'power/'+'power_GyrY_'+str(samp),self._power_GyrY)
        np.savez(p_path+'power/'+'power_GyrZ_'+str(samp),self._power_GyrZ)
        
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def CalcKLD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_VS_'+Sarray2,self._kld)
        
    def ShowKLD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
        
    def CalcJSD(self,Sarray1,Sarray2,samp):
        dist1 = self._GetPower(Sarray1,samp)
        dist2 = self._GetPower(Sarray2,samp)
        
        self._jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2,self._jsd)
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_VS_'+Sarray2+'.npz')['arr_0']
        print Sarray1+'_VS_'+Sarray2+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
        
    def ShowFFT(self,name):
        data = self._String2Var(name)
        print data

In [ ]:
test = RecordData()
test.ImportCSV('west.csv')
#test.CalcFFT(512)
#test.CalcPower(512)
test.CalcKLD('power_AccX','power_AccX',16)
test.ShowKLD('power_AccX','power_AccX',16)

test.CalcJSD('power_AccX','power_AccX',16)
test.ShowJSD('power_AccX','power_AccX',16)

#print test._power_AccX.shape
#d=test._Query('AccX',[test._Time[0],test._Time[0]+datetime.timedelta(seconds=0.16)])
#fftpack.fft(test._AccX)
#print test._Time[0]
#print type(test._Time[0])
#print test._Time[0]+np.timedelta64(160,'ms')
#np.where(test._Time==np.datetime64('2016-05-20T18:06:15.908000000') )
#min(test._Time)> max(test._Time)
#test.ShowFFT('fft_AccX')
#test.ShowFFT('fft_AccY')
#test.ShowFFT('fft_AccZ')
#test.ShowFFT('fft_GyrX')
#test.ShowFFT('fft_GyrY')
#test.ShowFFT('fft_GyrZ')

__class__
power_AccX_VS_power_AccXKullback–Leibler Divergence
shape =(121801,)
[ 0.          0.26283912  0.16866306 ...,  0.11657819  0.20916036  0.        ]


In [ ]:
data = pd.read_csv("Conv-west-ags.csv",encoding="Shift-JIS")
data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
data.AccX = data.AccX*0.1*0.001
data.AccY = data.AccY*0.1*0.001
data.AccZ = data.AccZ*0.1*0.001

data.GyrX = data.GyrX*0.1*0.001
data.GyrY = data.GyrY*0.1*0.001
data.GyrZ = data.GyrZ*0.1*0.001

data.Time=pd.to_datetime(data.Time)

data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
AccX=data.AccX

In [ ]:
2784/8

In [526]:
x=np.array( [[1,2,3],[4,5,6]])
y=np.array( [[6,5,4],[3,2,1]])

f=lambda x,y: np.sqrt(x**2+y**2)
f(x,y)

array([[ 6.08276253,  5.38516481,  5.        ],
       [ 5.        ,  5.38516481,  6.08276253]])

In [550]:
4.0/15

0.26666666666666666